In [1]:
import numpy as np 
import pandas as pd
import json 
from typing import List
from datetime import datetime
import os

import snowflake
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import sproc, col, lit, count

In [3]:
#Build session object: 
accountname = 'xe85544.east-us-2.azure'
username = 'kx'
password = 'Snowflake1'
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "KX",
    "schema": "BRUNO",
    "warehouse": "KX"
}

session = Session.builder.configs(connection_parameters).create()

### Creating the Ticker List: 

In this section, we will output a list of lists that contain tickers that will be fed into a Sproc. 

This algo looks to fill buckets based on a row max input.

In [5]:
trades_df = session.table('trades')
quotes_df = session.table('quotes')

trades_agg = trades_df.select(col('"Symbol"')).group_by('"Symbol"')\
            .agg(count('"Symbol"').alias('Record_Count')).to_pandas()

quotes_agg = quotes_df.select(col('"Symbol"')).group_by('"Symbol"')\
            .agg(count('"Symbol"').alias('Record_Count')).to_pandas()

records_df = trades_agg.merge(quotes_agg, how = 'inner', left_on = 'Symbol', right_on = 'Symbol')

records_df['SUM'] = records_df['RECORD_COUNT_x'] + records_df['RECORD_COUNT_y']
records_df.sort_values(by='SUM', inplace=True)
records_df.head(5)

,Symbol,RECORD_COUNT_x,RECORD_COUNT_y,SUM
3025,CHNG,1,2,3
5134,MTEST,2,6,8
2739,RSXJ,7,6,13
1683,RSX,8,6,14
6091,PTEST,7,10,17


In [6]:
def group_stocks(df, ROW_MAX):
    groups = []
    group_total = 0
    group_symbols = set()
    for i, row in df.iterrows():
        symbol, value = row['Symbol'], row['SUM']
        if group_total + value <= ROW_MAX:
            group_total += value
            group_symbols.add(symbol)
        else:
            groups.append(list(group_symbols))
            group_total = value
            group_symbols = {symbol}
    groups.append(list(group_symbols))
    return groups

In [52]:
from snowflake.snowpark import *
from snowflake.snowpark.functions import *

In [53]:
w = Window.partition_by('"Symbol"').order_by('"TTime"')

In [83]:
select_columns = ['"TTime"', '"Symbol"', '"Trade Volume"', '"Trade Price"', "r2"]
df2 = session.table('trades').select(col('*'), row_number().over(w).alias("r2")).select(select_columns)

In [84]:
select_columns = ['"TTime"', '"Symbol"', '"Bid_Size"', '"Bid_Price"', '"Offer_Size"', '"Offer_Price"', "r1"]
df1 = session.table('quotes').select(col('*'), row_number().over(w).alias("r1")).select(select_columns)


In [56]:
df1

,TTime,Symbol,Bid_Size,Bid_Price,Offer_Size,Offer_Price,R1
0,2022-10-03 12:53:21.422509259,ADP,1.0,230.10,1.0,230.35,821686
1,2022-10-03 12:53:21.422690699,ADP,2.0,230.19,2.0,230.35,821687
2,2022-10-03 12:53:21.503609261,ADP,1.0,230.19,2.0,230.35,821688
3,2022-10-03 12:53:21.513030476,ADP,3.0,230.17,1.0,230.31,821689
4,2022-10-03 12:53:21.513030554,ADP,3.0,230.17,1.0,230.32,821690
5,2022-10-03 12:53:21.513745903,ADP,2.0,230.17,1.0,230.32,821691
6,2022-10-03 12:53:21.514131848,ADP,1.0,216.02,1.0,243.02,821692
7,2022-10-03 12:53:21.514176345,ADP,1.0,216.02,1.0,230.28,821693
8,2022-10-03 12:53:21.514569250,ADP,1.0,230.18,2.0,230.35,821694
9,2022-10-03 12:53:21.515294845,ADP,1.0,230.18,1.0,230.28,821695


In [57]:
df2

,TTime,Symbol,Trade Volume,Trade Price,R2
0,2022-10-03 14:02:00.408371456,BFH,10,32.11,10631
1,2022-10-03 14:02:00.408376576,BFH,7,32.11,10632
2,2022-10-03 14:02:00.408376576,BFH,100,32.11,10633
3,2022-10-03 14:02:00.408376576,BFH,2,32.11,10634
4,2022-10-03 14:02:00.408386560,BFH,98,32.11,10635
5,2022-10-03 14:02:00.408396544,BFH,100,32.11,10636
6,2022-10-03 14:02:00.408722176,BFH,30,32.11,10637
7,2022-10-03 14:02:00.408722176,BFH,10,32.11,10638
8,2022-10-03 14:02:00.408737280,BFH,70,32.11,10639
9,2022-10-03 14:02:00.408762368,BFH,1,32.11,10640


In [88]:
df_zipped = df1.join(
    df2,
    (df1["R1"] == df2["R2"] ) & (df1['"Symbol"'] == df2['"Symbol"']),
    how="outer",
    lsuffix='_quote',
    rsuffix='_trade'
).select(col('*')).limit(1000).to_pandas()





In [90]:
df_zipped

,TTime_quote,Symbol_quote,Bid_Size,Bid_Price,Offer_Size,Offer_Price,R1,TTime_trade,Symbol_trade,Trade Volume,Trade Price,R2
0,2022-10-03 09:30:18.752406016,A,1.0,121.98,1.0,123.52,918,2022-10-03 09:52:28.082626304,A,21.0,123.2800,918.0
1,2022-10-03 09:30:26.106880256,A,1.0,121.98,1.0,123.51,957,2022-10-03 09:53:41.765997568,A,100.0,123.0300,957.0
2,2022-10-03 09:30:32.535921920,A,3.0,122.48,1.0,123.03,986,2022-10-03 09:54:06.821563648,A,26.0,123.0500,986.0
3,2022-10-03 09:33:13.779725056,A,1.0,121.40,1.0,123.11,2055,2022-10-03 10:12:56.084943872,A,1.0,124.2400,2055.0
4,2022-10-03 09:33:31.008375040,A,1.0,121.64,1.0,122.89,2151,2022-10-03 10:13:36.197161984,A,14.0,124.3600,2151.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022-10-03 10:05:07.679834112,A,2.0,123.95,2.0,124.14,22498,2022-10-03 15:57:13.554260736,A,34.0,126.3871,22498.0
996,2022-10-03 10:05:08.873806336,A,1.0,123.96,1.0,124.27,22714,2022-10-03 15:58:01.925624576,A,200.0,126.3100,22714.0
997,2022-10-03 10:05:27.202323456,A,3.0,123.98,1.0,124.11,22945,2022-10-03 15:58:41.520054528,A,5.0,126.2900,22945.0
998,2022-10-03 10:05:33.003004928,A,2.0,123.99,1.0,124.20,23125,2022-10-03 15:59:08.612731136,A,26.0,126.4200,23125.0


In [ ]:
df_zipped.join_table_function(as_of.over(partition_by = "symbol"))


In [7]:
ticker_buckets = group_stocks(records_df, 10_000_000)

In [8]:
len(ticker_buckets)

173

In [12]:
ticker_buckets[0]

['PCTT U',
 'CHG',
 'MBAC U',
 'KEY PRL',
 'BANF P',
 'CLRC',
 'IACC',
 'OKYO',
 'KIM PRM',
 'GRU',
 'PFIE',
 'LMST',
 'GFOR',
 'DTRT',
 'BTTX',
 'GNFT',
 'NCPL W',
 'JFBR W',
 'AUBN',
 'RILY G',
 'MNSB',
 'NBST',
 'IBDN',
 'PWFL',
 'OXUS',
 'DISA W',
 'ACR PRC',
 'PACX',
 'MFUL',
 'DMAT',
 'AGRX',
 'MLAI',
 'GLBS',
 'NS PRA',
 'IMPP P',
 'GRNQ',
 'ADFI',
 'BMAC WS',
 'CRBP',
 'PETV W',
 'LMFA',
 'DCTH',
 'MIO',
 'MLVF',
 'ALSA W',
 'FRC PRI',
 'LOAN',
 'PSB PRY',
 'LHC WS',
 'ML WS',
 'IHY',
 'RACB',
 'ANPC',
 'HCTI',
 'FXCO R',
 'MHF',
 'ACRO WS',
 'GRNB',
 'AHT PRF',
 'FPAC',
 'QLI',
 'IHIT',
 'CMRE PRE',
 'BBAI WS',
 'LUCY',
 'MAAX',
 'CURI W',
 'MJXL',
 'AKU',
 'CREG',
 'ORTX',
 'CLGN',
 'FTPA U',
 'GSRM',
 'ESHY',
 'ONBP P',
 'BEAT W',
 'ENVB',
 'ATAX',
 'MBTC R',
 'HYMC W',
 'FPEI',
 'MFA PRB',
 'BNOX',
 'OXAC W',
 'GPJA',
 'LOCC',
 'CLRM W',
 'FGF',
 'OFED',
 'FHS',
 'JG',
 'DLNG',
 'ADRA WS',
 'PGSS U',
 'RXRA',
 'CFFS',
 'EPHY W',
 'ICAP',
 'GHAC',
 'AIMA U',
 'ECCC',
 'BWSN'

### Create and run Sproc using multi-cluster warehouses: 

Steps: 
1. create a list of list of tickers to pass into a sproc
2. create an SPROC that has an input list as an argument, that contains the respective tickers to filter by, and calls the main logic

In [10]:
license = 'Y8NTo4WkoyVFpNkz0oikQnrkIJbISSogiCwXxiUY/idrTFbSW9NHlidzlg1GDZYsoyeWTCzSvfkOliez/uOGvRem6fmbR9JkDEaTVpZjvVs7NltmlqSmGFvtNg2WNsuINhgs2UcXR/5Gh5vLLKMH/Nnx/DOIMyayRmMGJvEGU4i5hrJG'

In [11]:
%%time

@sproc(packages=['snowflake-snowpark-python', 'numpy', 'pandas', 'pyarrow'],
       imports=['@kx/pykx-package.zip', '@kx/01_interval_returns.q_'],
       name='ms_test_sproc', 
       is_permanent = True, 
       stage_location= '@KX', 
       replace = True)
def ms_test_sproc(session: Session, tickers: List[str]) -> None:
    """Takes in a list of tickers that we will be filtering on"""
    import base64, json, os, shutil, sys, time, zipfile
   
    ## CUSTOM SETUP OF PyKX (won't be needed when PyKX is available on Anaconda)
    tstart = time.perf_counter()
    if not os.path.exists('/tmp/k4.lic'):
        import_dir = sys._xoptions.get("snowflake_import_directory")
        lib = 'pykx-package.zip'
        lib_path = os.path.join(import_dir, lib)
        load_path = '/tmp/pykx-package/'

        if os.path.exists('/tmp/pykx-package'):
            shutil.rmtree('/tmp/pykx-package')
            time.sleep(0)
        with zipfile.ZipFile(lib_path, 'r') as zip_ref:
            zip_ref.extractall('/tmp/pykx-package')

        sys.path.insert(0, '/tmp/pykx-package')
        
        shutil.copyfile(os.path.join(import_dir, '01_interval_returns.q_'), '/tmp/01_interval_returns.q_')

        with open('/tmp/k4.lic', 'wb') as f:
            f.write(base64.b64decode(license))
    tend = time.perf_counter()
    tpykxinstall = tend - tstart
    ## END

    tstart = time.perf_counter()
    os.environ['QHOME'] = ''
    os.environ['QLIC'] = '/tmp'

    os.environ['PYKX_LOAD_PYARROW_UNSAFE'] = '1'
    os.environ['PYKX_NOQCE'] = '1'
    os.environ['SKIP_UNDERQ'] = '1'
    os.environ['IGNORE_QHOME'] = '1'
    os.environ['QARGS'] = '--licensed'
    
    import pykx as kx
    tend = time.perf_counter()
    tpykximport = tend - tstart
    
    tstart = time.perf_counter()
    select_columns = ['"TTime"', '"Symbol"', '"Trade Volume"', '"Trade Price"']
    trades = session.table('trades').select(select_columns)\
                    .filter(col('"Symbol"').isin(tickers))\
                    .order_by('"TTime"').to_pandas()                 # 1
    trades.columns = ['time', 'symbol', 'volume', 'price']
    tend = time.perf_counter()
    tloadtrades = tend - tstart
    
    tstart = time.perf_counter()
    kx.q['trades'] = trades
    del trades
    tend = time.perf_counter()
    ttoqtrades = tend - tstart

    tstart = time.perf_counter()
    select_columns = ['"TTime"', '"Symbol"', '"Bid_Size"', '"Bid_Price"', '"Offer_Size"', '"Offer_Price"']
    quotes = session.table('quotes').select(select_columns)\
                    .filter(col('"Symbol"').isin(tickers))\
                    .order_by('"TTime"').to_pandas()                 # 2
    quotes.columns = ['time', 'symbol', 'ask_volume', 'ask_price', 'bid_volume', 'bid_price']
    tend = time.perf_counter()
    tloadquotes = tend - tstart
    
    tstart = time.perf_counter()
    kx.q['quotes'] = quotes
    del quotes
    tend = time.perf_counter()
    ttoqquotes = tend - tstart

    tstart = time.perf_counter()
    kx.q('\l /tmp/01_interval_returns.q_')  # Run a locked q script to apply the business logic             # 3
    tend = time.perf_counter()
    tqalgo = tend - tstart
    
    tstart = time.perf_counter()
    output = kx.q('t').pd()
    tend = time.perf_counter()
    ttopd = tend - tstart
    
    tstart = time.perf_counter()
    session.write_pandas(output, 'MS_OUTPUT', overwrite=False, auto_create_table=True, chunk_size=1000000, compression='snappy')        # 4
    tend = time.perf_counter()
    tout = tend - tstart
    
    return None

CPU times: user 141 ms, sys: 15.9 ms, total: 157 ms
Wall time: 32.7 s


In [13]:
import time
from joblib import Parallel, delayed
start = datetime.now()

#instantiate queries
query_ids = []

def execute_sproc(job):
    
    conn = snowflake.connector.connect(
        account="xe85544.east-us-2.azure",
        user="kx",
        password = 'Snowflake1',
        role="ACCOUNTADMIN",  # optional
        warehouse="KX",
        database="KX",
        schema="BRUNO",
    )
    cur = conn.cursor()
    cur.execute_async(f'CALL MS_TEST_SPROC({job})')
    return cur.sfqid

results = Parallel(n_jobs=-1)(delayed(execute_sproc)(job) for job in ticker_buckets)

query_ids.extend(results)

end = datetime.now()
print(end-start)

0:00:09.244673


In [14]:
conn = snowflake.connector.connect(
    account="xe85544.east-us-2.azure",
    user="kx",
    password = 'Snowflake1',
    role="ACCOUNTADMIN",  # optional
    warehouse="kx",  # medium snowpark-optimized
    database="KX",
    schema="BRUNO",
)
cur = conn.cursor()

t = cur.execute(f'''
SELECT
    MAX(END_TIME) AS end_time
FROM
    SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
WHERE QUERY_ID in {tuple(query_ids)}
''').fetch_pandas_all()

In [15]:
t['END_TIME'].values[0] - pd.Timestamp(start) - pd.Timedelta(hours=4) #timezone adjust

Timedelta('0 days 00:01:37.837763')